In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# StageLoader: products_Stage Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_products` (
         `bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`product_id`
        ,`product_name`
        ,`unit_price`
        ,`product_details`
        ,`product_image`
        ,`image_mime_type`
        ,`image_filename`
        ,`image_charset`
        ,`image_last_updated`
    )
    SELECT
         {bg_loadtimestamp} AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`product_id` AS `product_id`
        ,`bg_source`.`product_name` AS `product_name`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`product_details` AS `product_details`
        ,`bg_source`.`product_image` AS `product_image`
        ,`bg_source`.`image_mime_type` AS `image_mime_type`
        ,`bg_source`.`image_filename` AS `image_filename`
        ,`bg_source`.`image_charset` AS `image_charset`
        ,`bg_source`.`image_last_updated` AS `image_last_updated`
    FROM `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_products_source` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['reloadtarget_{marc_databricks_initiative#stage#database_name}_{marc_databricks_initiative#stage#schema_name}_stg_st_products'] = operation_metrics

except Exception as e:
    info('STG_ST_products_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
